In [11]:
import numpy as np
import pandas as pd
import os
import subprocess
import math
import matplotlib.pyplot as plt

import glob

import csv

In [12]:
def ProcessSAMdat_single_redshift(path_to_SAM, Nsubvols, sought_z, fieldswanted, gal_or_halo):
    g_colnames = ['halo_index', 'birthhaloid', 'roothaloid', 'redshift', 'sat_type',
                  'mhalo', 'm_strip', 'rhalo', 'mstar', 'mbulge', 'mstar_merge', 'v_disk',
                  'sigma_bulge', 'r_disk', 'r_bulge', 'mcold', 'mHI', 'mH2', 'mHII', 'Metal_star',
                  "Metal_cold", 'sfr', 'sfrave20myr', 'sfrave100myr', 'sfrave1gyr',
                  'mass_outflow_rate', 'metal_outflow_rate', 'mBH', 'maccdot', 'maccdot_radio',
                  'tmerge', 'tmajmerge', 'mu_merge', 't_sat', 'r_fric', 'x_position',
                  'y_position', 'z_position', 'vx', 'vy', 'vz']
    h_colnames = ['halo_index', 'halo_id', 'roothaloid', 'orig_halo_ID', 'redshift', 'm_vir', 'c_nfw',
                  'spin', 'm_hot', 'mstar_diffuse', 'mass_ejected', 'mcooldot',
                  'maccdot_pristine', 'maccdot_reaccrete', 'maccdot_metal_reaccrete',
                  'maccdot_metal', 'mdot_eject', 'mdot_metal_eject', 'maccdot_radio',
                  'Metal_hot', 'Metal_ejected', 'snap_num']
    g_header_rows = []
    for i in range(0, len(g_colnames)):
        g_header_rows.append(i)
    h_header_rows = []
    for i in range(0, len(h_colnames)):
        h_header_rows.append(i)

    input_path=path_to_SAM

    if type(fieldswanted) == list:
        print(type(fieldswanted))
    else:
        return "Fieldswanted should be a list with the fields you want as strings!"

    All_halos=np.zeros((1,len(fieldswanted)))
    if gal_or_halo=="gal":
        
        checknums=0
        
        if Nsubvols == 1:
            
            for x_i in np.arange(0, 1,1):

                for x_j in np.arange(0,1,1):
                
                    for x_k in np.arange(0,1,1):
                    
                        galprop = pd.read_csv('{}/{}_{}_{}/galprop_0-99.dat'.format(input_path, x_i, x_j, x_k),
                                              delimiter=' ', skiprows=g_header_rows, names=g_colnames)
                        # print('galprop read for ',x_i, x_j, x_k,' shape:', galprop.shape)
                        current_galprops=galprop[fieldswanted[:]].to_numpy()
                        #print('For subvolume ',x_i,x_j,x_k, current_galprops.shape)
                        unique_redshifts=set(current_galprops[:,0])
                        unique_redshifts = np.array(sorted(unique_redshifts))
                        # print(unique_redshifts)
                        idx = (np.abs(unique_redshifts - sought_z)).argmin()
                        current_galprops_z=current_galprops[np.where(current_galprops[:,0][:]==unique_redshifts[idx])[0],:]
                        # print(current_galprops_z.shape)
                        checknums=checknums+len(current_galprops_z)
                        All_halos=np.concatenate((All_halos,current_galprops_z))

        if Nsubvols == 8:
            
            for x_i in np.arange(0, 2,1):

                for x_j in np.arange(0,2,1):
                
                    for x_k in np.arange(0,2,1):
                        
                        galprop = pd.read_csv('{}/{}_{}_{}/galprop_0-99.dat'.format(input_path, x_i, x_j, x_k),
                                              delimiter=' ', skiprows=g_header_rows, names=g_colnames)
                        # print('galprop read for ',x_i, x_j, x_k,' shape:', galprop.shape)
                        current_galprops=galprop[fieldswanted[:]].to_numpy()
                        print('For subvolume ',x_i,x_j,x_k, current_galprops.shape)
                        unique_redshifts=set(current_galprops[:,0])
                        unique_redshifts = np.array(sorted(unique_redshifts))
                        # print(unique_redshifts)
                        idx = (np.abs(unique_redshifts - sought_z)).argmin()
                        current_galprops_z=current_galprops[np.where(current_galprops[:,0][:]==unique_redshifts[idx])[0],:]
                        # print(current_galprops_z.shape)
                        checknums=checknums+len(current_galprops_z)
                        All_halos=np.concatenate((All_halos,current_galprops_z))
        print(All_halos.shape, checknums)
        return All_halos
    
    elif gal_or_halo=="halo":
        checknums2=0
        for x_i in np.arange(0,Nsubvols,1):   #is it here? the problem is this Nsubvols thing? like it's trying to go all the way to 8?
            for x_j in np.arange(0,Nsubvols,1):
                for x_k in np.arange(0,Nsubvols,1):
                    haloprop = pd.read_csv('{}/{}_{}_{}/haloprop_0-99.dat'.format(input_path, x_i, x_j, x_k),
                                           delimiter=' ', skiprows=h_header_rows, names=h_colnames)
                    current_haloprops=haloprop[fieldswanted[:]].to_numpy()
                    print('For subvolume ',x_i,x_j,x_k, current_haloprops.shape)
                    unique_redshifts=set(current_haloprops[:,0])
                    unique_redshifts = np.array(sorted(unique_redshifts))
                    idx = (np.abs(unique_redshifts - sought_z)).argmin()
                    current_haloprops_z=current_haloprops[np.where(current_haloprops[:,0][:]==unique_redshifts[idx])[0],:]
                    # print(current_haloprops_z.shape)
                    checknums2=checknums2+len(current_haloprops_z)
                    All_halos=np.concatenate((All_halos,current_haloprops_z))
        print(All_halos.shape, checknums2)
        return All_halos[1:,:]
    else:
        print("gal_or_halo need to be a string, either 'gal' or 'halo', to get galprop or haloprop respectively. Make sure the fields you want are actually reflected!")
        print("Column names of galprop file: ", g_colnames)
        print("Column names of haloprop file: ", h_colnames)
        return All_halos

In [13]:


def run_for_all_sims(start, stop):
    
    fields_wanted = ['redshift','mhalo']
    
    bins = np.geomspace(1, 10**7, num = 20)
    
    
    np.diff(np.log10(bins))

    diff = np.diff(np.log10(bins))

    bin_width = diff[0]
        

    bins =list(bins)
        
        
    log_bin = [(j*(10**9)) for j in bins]
        
        
    rows = {
    
    "bins": log_bin
            
    }
    
    global hmf_df
    
    hmf_df = pd.DataFrame(rows)
    
    counter = 0
        
        
        
    
    for i in range(start,stop):
        
        
    
        mypath = '/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_' + str(i) + '/sc-sam'
        
        
        print(mypath)
        
        #count subvolumes 
        subvol_dirs = glob.glob("%s/*_*_*" % mypath)
        
        N_subvol = (len(subvol_dirs))
        print(N_subvol)
        
        #load sim
        sim_rshiftzero = ProcessSAMdat_single_redshift(mypath, N_subvol, 0, fields_wanted, 'gal') #returns redshift in first column and halo mass in second column
        
        
        
        masses_selected  = (sim_rshiftzero[:, 1])
        
        (n, bins, patches) = plt.hist(masses_selected, np.geomspace(1, 10**7, num = 20))
       
        n = list(n)

        norm_n = [j/((100**3)*bin_width) for j in n]

        norm_n.append(0)
        

        hmf_df.insert(1, str("count" + str(i)), norm_n)
        
        print(str("sim num " + str(i)))
        
        
        hmf_df.to_csv("hmf_csv")
        
        

                
        
                
                
                
                
            
            

        
        

        
        
        
    hmf_df["mean"] = hmf_df.iloc[:,1:].mean(axis=1)

    
    hmf_df["std dev"] = np.std(hmf_df.iloc[:,1:-1], axis = 1) #is this right?
    
    hmf_df.to_csv("hmf_csv")
    
    
    
   

In [14]:
run_for_all_sims(1, 1001)

/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_1/sc-sam
1
<class 'list'>
(135120, 2) 135119
sim num 1
/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_2/sc-sam
8
<class 'list'>
For subvolume  0 0 0 (1673163, 2)
For subvolume  0 0 1 (1481890, 2)
For subvolume  0 1 0 (2471335, 2)
For subvolume  0 1 1 (2100958, 2)
For subvolume  1 0 0 (1359648, 2)
For subvolume  1 0 1 (835203, 2)
For subvolume  1 1 0 (1922654, 2)
For subvolume  1 1 1 (1079192, 2)
(153179, 2) 153178
sim num 2
/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_3/sc-sam
1
<class 'list'>
(130034, 2) 130033
sim num 3
/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_4/sc-sam
8
<class 'list'>
For subvolume  0 0 0 (1336531, 2)
For subvolume  0 0 1 (1853615, 2)
For subvolume  0 1 0 (1359929, 2)
For subvolume  0 1 1 (1376387, 2)
For subvolume  1 0 0 (2088439, 2)
For subvolume  1 0 1 (2024693, 2)
For subvolume  1 1 0 (1270913, 2)
For subvolume  1 1 1 (1891899, 2)
(183941, 2) 183940
sim num 4
/mnt/ceph/users/camels/PUBLIC_RELEASE/SCSAM/LH_5/

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f887fd318c8> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#hmf_df

In [ ]:
plt.rcParams["figure.figsize"] = (16,11)

plt.plot(hmf_df["bins"], hmf_df.iloc[:,1:-2])

plt.plot(hmf_df["bins"], hmf_df["mean"], color = 'black', label = "mean")

plt.errorbar(hmf_df["bins"], hmf_df["mean"], yerr= hmf_df["std dev"], fmt='-o', color = "black")

plt.legend(fontsize = 18)

plt.xlabel(r'$\log M_{halo}, M solar $', fontsize = 14)

plt.ylabel(r'$\ dn/logM (Mpc^{-3})$', fontsize = 14)

plt.xscale('log')

plt.yscale('log')

plt.xticks(fontsize = 18)

plt.yticks(fontsize = 18)

plt.show()
    